# Analiza podatkov
V sledečem zvezku se nahaja analiza podatkov iz spletne strani [Musescore](https://musescore.com/sheetmusic).

V _prvem delu_ bom izvedla preprosto analizo pridobljenih podatkov. 
Zanimalo me bo, kako se posamezni parametri povezujejo s popularnostjo skladbe ter povezave med parametri.
Konkretno želim odgovoriti na naslednja vprašanja:
* Katerih skladb je največ nasploh in za specifične kategorije?
* Katere so najpopularnejše?
* Kakšna je povezava med popularnostjo in številčnostjo skladb glede na kategorije?
* Kakšne so lasnosti skladb, ki spadajo v Klasično glasbo, v primerjavi z ostalimi?


V _drugem delu_ se bom lotila nekoliko zahtevnejše analize.
Zanimalo me bo, če obstaja recept za uspešno objavo - pri tem bom uporabila pridobljene informacije iz prvega dela; hkrati pa bom izdelala napovedovalec popularnosti skladbe glede na parametre, kot je žanr, inštrument in podobno.

Zanimala pa me bo tudi časovna slika podatkov. Tako bom podatke razdelila na smiselne časovne intervale in znotraj teh poiskala parametre, kot je povprečna zvrst, dolžina itn. objavljenih skladb v tem obdobju; prav tako pa bom po zastopanosti v tem obdobju uredila originale. Iskala bom obdobja večjih sprememb v podatkih in poskusila najti odgovor, zakaj je do njih prišlo. Zanimala me bo dominantnost posameznih žanrov in inštrumentov, z analizo žanrov pa bom tudi ustvarila sliko glasbenih okusov skozi čas.  
Glede na rezultate bom ustvarila tudi predpostavke za spremembe trendov v prihodnosti.  
Dobljene rezultate bom tudi vizualno predstavila s smiselnimi grafi.
Ugotovitve o časovnih trendih bom dodala svojemu napovedovalcu popularnosti.

## Uvoz podatkov

In [2]:
# Naložimo potrebne knjižnjice
import pandas as pd
import os.path 
import ast
import seaborn

# Nastavimo pravilen prikaz grafov
%matplotlib inline

# Nastavimo izpisovanje po 6 vrstic, saj delamo z ogromno količino podatkov, in bi bile razpredelnice sicer prevelike.
pd.options.display.max_rows = 6

In [3]:
# naložimo osnovno razpredelnico, s katero bomo delali
music = pd.read_csv(os.path.join('obdelani-podatki','all_music.csv'), index_col="music_id")
music

,title,rating,votes,ensemble,instrument,key,date,duration,public_domain,favorites,difficulty,pages,views,parts,link
music_id,,,,,,,,,,,,,,,
4197961,Merry-Go-Round of Life: Howl's Moving Castle P...,4.77,47676,Solo Piano,['Piano'],"['B♭ major', 'G minor']","Jul 05, 2017",05:15,NE,181934,Intermediate,7,4589622,1,https://musescore.com/user/16006641/scores/419...
3291706,River Flows In You,4.68,45503,Solo Piano,['Piano'],"['A major', 'F♯ minor']","Jan 24, 2017",02:47,NE,207886,Intermediate,4,5975850,1,https://musescore.com/user/12461571/scores/329...
1019991,Canon in D,4.71,33989,Solo Piano,['Piano'],"['D major', 'B minor']","Jul 01, 2015",04:05,DA,137361,Intermediate,6,3275886,1,https://musescore.com/user/1809056/scores/1019991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4766136,Boogie Woogie and Blues Licks,4.62,323,Solo Piano,['Piano'],"['C major', 'A minor']","Oct 21, 2017",01:07,DA,1633,Intermediate,2,25978,1,https://musescore.com/user/39215063/scores/476...
4926407,Cruel Angel's Thesis - Piano Cover by Theishter,4.86,323,Solo Piano,['Piano'],"['E♭ major', 'C minor']","Feb 07, 2018",01:33,NE,1732,Advanced,3,49784,1,https://musescore.com/user/27635868/scores/492...
5087038,Vincent Starry Starry Night,4.67,323,Solo Piano,['Piano'],"['G major', 'E minor']","May 08, 2018",04:08,NE,1200,Intermediate,4,53453,1,https://musescore.com/user/28424340/scores/508...


Tabelo želimo nekoliko popraviti. Tonalitete bi radi v obliki enega samega niza; pri tem upoštevamo, da je vedno podan par komplementnih tonalitet.

In [4]:
# Podatki v tabeli le izgledajo kot seznami, vendar so v resnici tipa string. To popravimo z uporabo knjižnjice ast
music["key"] = music["key"].apply(ast.literal_eval)

# Funkcija, ki združuje dva elementa seznama
def concatenate_key(key_list):
    return ', '.join(key_list)

# To funkcijo uporabimo na naši tabeli
music['key'] = music['key'].apply(concatenate_key)

music

,title,rating,votes,ensemble,instrument,key,date,duration,public_domain,favorites,difficulty,pages,views,parts,link
music_id,,,,,,,,,,,,,,,
4197961,Merry-Go-Round of Life: Howl's Moving Castle P...,4.77,47676,Solo Piano,['Piano'],"B♭ major, G minor","Jul 05, 2017",05:15,NE,181934,Intermediate,7,4589622,1,https://musescore.com/user/16006641/scores/419...
3291706,River Flows In You,4.68,45503,Solo Piano,['Piano'],"A major, F♯ minor","Jan 24, 2017",02:47,NE,207886,Intermediate,4,5975850,1,https://musescore.com/user/12461571/scores/329...
1019991,Canon in D,4.71,33989,Solo Piano,['Piano'],"D major, B minor","Jul 01, 2015",04:05,DA,137361,Intermediate,6,3275886,1,https://musescore.com/user/1809056/scores/1019991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4766136,Boogie Woogie and Blues Licks,4.62,323,Solo Piano,['Piano'],"C major, A minor","Oct 21, 2017",01:07,DA,1633,Intermediate,2,25978,1,https://musescore.com/user/39215063/scores/476...
4926407,Cruel Angel's Thesis - Piano Cover by Theishter,4.86,323,Solo Piano,['Piano'],"E♭ major, C minor","Feb 07, 2018",01:33,NE,1732,Advanced,3,49784,1,https://musescore.com/user/27635868/scores/492...
5087038,Vincent Starry Starry Night,4.67,323,Solo Piano,['Piano'],"G major, E minor","May 08, 2018",04:08,NE,1200,Intermediate,4,53453,1,https://musescore.com/user/28424340/scores/508...


Popraviti želimo tudi stolpec "duration". Podatki v njem so tipa string, nam pa bo primernejši za uporabo tip int, s katerim bomo lažje pretvarjali dane podatke v timedelta format.

In [5]:
# Pretvorba v sekunde, ki ulovi primere, ko nimamo podatkov
def convert_to_seconds(duration):
    if pd.isna(duration) or not isinstance(duration, str):
        return None
    parts = duration.split(':')
    if len(parts) == 2:
        try:
            minutes, seconds = map(int, parts)
            return minutes * 60 + seconds
        except ValueError:
            return None
    return None

music['duration_seconds'] = music['duration'].apply(convert_to_seconds)
music

,title,rating,votes,ensemble,instrument,key,date,duration,public_domain,favorites,difficulty,pages,views,parts,link,duration_seconds
music_id,,,,,,,,,,,,,,,,
4197961,Merry-Go-Round of Life: Howl's Moving Castle P...,4.77,47676,Solo Piano,['Piano'],"B♭ major, G minor","Jul 05, 2017",05:15,NE,181934,Intermediate,7,4589622,1,https://musescore.com/user/16006641/scores/419...,315.0
3291706,River Flows In You,4.68,45503,Solo Piano,['Piano'],"A major, F♯ minor","Jan 24, 2017",02:47,NE,207886,Intermediate,4,5975850,1,https://musescore.com/user/12461571/scores/329...,167.0
1019991,Canon in D,4.71,33989,Solo Piano,['Piano'],"D major, B minor","Jul 01, 2015",04:05,DA,137361,Intermediate,6,3275886,1,https://musescore.com/user/1809056/scores/1019991,245.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4766136,Boogie Woogie and Blues Licks,4.62,323,Solo Piano,['Piano'],"C major, A minor","Oct 21, 2017",01:07,DA,1633,Intermediate,2,25978,1,https://musescore.com/user/39215063/scores/476...,67.0
4926407,Cruel Angel's Thesis - Piano Cover by Theishter,4.86,323,Solo Piano,['Piano'],"E♭ major, C minor","Feb 07, 2018",01:33,NE,1732,Advanced,3,49784,1,https://musescore.com/user/27635868/scores/492...,93.0
5087038,Vincent Starry Starry Night,4.67,323,Solo Piano,['Piano'],"G major, E minor","May 08, 2018",04:08,NE,1200,Intermediate,4,53453,1,https://musescore.com/user/28424340/scores/508...,248.0


In [6]:
music.rename(columns={"instrument":"all_instruments"}, inplace=True)

Naložimo tudi pomožne razpredelnice, v katerih imamo dodatne podatke o avtorjih, uporabnikih, itd.

In [7]:
authors = pd.read_csv(os.path.join('obdelani-podatki','authors.csv'), index_col="author_id")
genres = pd.read_csv(os.path.join('obdelani-podatki','genres.csv'))
users = pd.read_csv(os.path.join('obdelani-podatki','users.csv'), index_col="user_id")
relacije_authors = pd.read_csv(os.path.join('obdelani-podatki','relacije_authors.csv'))
relacije_users = pd.read_csv(os.path.join('obdelani-podatki','relacije_users.csv'))

Pomagale nam bodo še naslednje tabele.

* tabela z urejenimi originali in informacijo o številu ponovitev

In [8]:
# Tabela s surovimi podatki
originals = relacije_authors.copy()

# Razvrstimo po skupinah glede na originalne avtorje in originalne naslove ter jih obdelamo
originals = originals.groupby(['original_author', 'original_title'])['music_id'].agg(list).reset_index()

# Dolžina seznama pri vsakem originalu nam pove, koliko vnosov mu pripada
originals['repetitions'] = originals['music_id'].apply(len)
originals


,original_author,original_title,music_id,repetitions
0,0,H0me Sw33t H0me,[5467064],1
1,7,Orange,[2497626],1
2,24,Mood,[6350578],1
...,...,...,...,...
1182,411822,Wednesday Play The Cello,[9144624],1
1183,447430,Cupid,[10367578],1
1184,452182,CMP,[6305471],1


* tabela s podatki o inštrumentih, ki niso več v obliki seznamov

In [9]:
# Podatki v tabeli le izgledajo kot seznami, vendar so v resnici tipa string. To popravimo z uporabo knjižnjice ast
music["all_instruments"] = music["all_instruments"].apply(ast.literal_eval)

# Izberemo le podatke o inštrumentih
instruments_lists = music[["all_instruments"]].copy().reset_index()

# Metoda explode pomaga pri razbitju listov na posamezne člene
instruments = instruments_lists.explode("all_instruments", ignore_index=True)

instruments.rename(columns={'all_instruments': 'instrument'}, inplace=True)

instruments

,music_id,instrument
0,4197961,Piano
1,3291706,Piano
2,1019991,Piano
...,...,...
1996,4766136,Piano
1997,4926407,Piano
1998,5087038,Piano


Nove tabele si shranimo.

In [10]:
instruments.to_csv(os.path.join('obdelani-podatki','instruments.csv'), index=False, encoding='utf-8')
originals.to_csv(os.path.join('obdelani-podatki','originals.csv'), index=False, encoding='utf-8')

## _Prvi del_: Osnovna analiza

Prvi del razdelimo na več vsebinskih sklopov. 
* **Količinska analiza**: Odgovorimo na vprašanje, kakšnih skladb je največ.
* **Popularnost**: Vzpostavimo merilo popularnosti in ga izračunamo. Ugotovimo, kakšne skladbe so najbolj popularne.
* **Klasična glasba**: Posebna obravnava klasične glasbe.
* **Zaključek**: Preverjanje veljavnosti hipotez in povzetek ugotovitev.

### Količinska analiza

### Popularnost

### Klasična glasba

### Zaključek